In [16]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType  # type: ignore

import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
import dolfinx
import meshio


In [17]:
filename="meshes/mesh.vtu"
msh = meshio.read(filename)

meshio.write("meshes/mesh.xdmf", msh)

In [18]:
filename="meshes/mesh.xdmf"
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, filename, "r") as xdmf:
       msh = xdmf.read_mesh(name="Grid")

In [19]:
# Used this to initially create meshes/mesh.vtu

# msh = mesh.create_rectangle(
#     comm=MPI.COMM_WORLD,
#     points=((0.0, 0.0), (2.0, 1.0)),
#     n=(32, 16),
#     cell_type=mesh.CellType.triangle,
# )

In [20]:
V = fem.functionspace(msh, ("Lagrange", 1))

In [21]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0),
)

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [22]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
# f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(5 * x[0])
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L = ufl.inner(f, v) * ufl.dx + ufl.inner(g, v) * ufl.ds

L = ufl.inner(g, v) * ufl.ds

In [ ]:
# There is a more manual method to solve this, similar to NGSolve, Heat_Equation_fenics.ipynb.

problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options_prefix="demo_poisson_",
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "ksp_error_if_not_converged": True},
)
uh = problem.solve()
assert isinstance(uh, fem.Function)

In [24]:
with io.XDMFFile(msh.comm, "output/poisson_rectangle_fenics.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)